In [65]:
import yfinance as yf
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates

pd.set_option('display.width', 1000)

In [66]:
symbol = 'IBM'
start_day = '1999-01-01'

In [67]:
ticker = yf.Ticker(symbol)
IBM_history = ticker.history(start=start_day)

IBM_history.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)

IBM_history['Prev_close'] = IBM_history['Close'].shift()
IBM_history['Rate'] = (IBM_history['Close'] - IBM_history['Prev_close']) / IBM_history['Prev_close'] * 100
IBM_history['Rate'][0] = (IBM_history['Close'][0] - IBM_history['Open'][0]) / IBM_history['Open'][0] * 100
IBM_history['Rate'] = IBM_history['Rate'].round(decimals=2)

IBM_history.reset_index(inplace=True)
IBM_history['Date'] = IBM_history['Date'].dt.strftime('%Y-%m-%d')
IBM_history.set_index(IBM_history['Date'], inplace=True)
IBM_history.drop(['Date', 'Prev_close'], axis=1, inplace=True)

c:\Users\ozing\miniconda3\envs\RL_trader\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [68]:
window_size = 20
data_rolling = IBM_history['Close'].rolling(window_size, min_periods=1)
IBM_history[f'MA_{window_size}'] = data_rolling.mean().astype(np.int32)

In [69]:
IBM_history['Upper Band'] = IBM_history[f'MA_{window_size}'] + (2 * data_rolling.std())
IBM_history['Lower Band'] = IBM_history[f'MA_{window_size}'] - (2 * data_rolling.std())
IBM_history['Upper Band'][0] = IBM_history[f'MA_{window_size}'][0] + (2 * np.std([IBM_history['Open'][0], IBM_history['Close'][0]]))
IBM_history['Lower Band'][0] = IBM_history[f'MA_{window_size}'][0] - (2 * np.std([IBM_history['Open'][0], IBM_history['Close'][0]]))

IBM_history['Upper Band'] = IBM_history['Upper Band'].round(2)
IBM_history['Lower Band'] = IBM_history['Lower Band'].round(2)

c:\Users\ozing\miniconda3\envs\RL_trader\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\ozing\miniconda3\envs\RL_trader\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [70]:
sp500 = yf.Ticker('^GSPC')
sp500_history = sp500.history(start=start_day)

sp500_history.reset_index(inplace=True)
sp500_history['Date'] = sp500_history['Date'].dt.strftime('%Y-%m-%d')
sp500_history.set_index(sp500_history['Date'], inplace=True)
sp500_history.drop(['Date'], axis=1, inplace=True)

sp500_history.rename({'Close':'SP500'}, axis=1, inplace=True)

In [71]:
sp500_history

,Open,High,Low,SP500,Volume,Dividends,Stock Splits
Date,,,,,,,
1999-01-04,1229.229980,1248.810059,1219.099976,1228.099976,877000000,0.0,0.0
1999-01-05,1228.099976,1246.109985,1228.099976,1244.780029,775000000,0.0,0.0
1999-01-06,1244.780029,1272.500000,1244.780029,1272.339966,986900000,0.0,0.0
1999-01-07,1272.339966,1272.339966,1257.680054,1269.729980,863000000,0.0,0.0
1999-01-08,1269.729980,1278.239990,1261.819946,1275.089966,937800000,0.0,0.0
...,...,...,...,...,...,...,...
2023-02-02,4158.680176,4195.439941,4141.879883,4179.759766,5624360000,0.0,0.0
2023-02-03,4136.689941,4182.359863,4123.359863,4136.479980,4694510000,0.0,0.0
2023-02-06,4119.569824,4124.629883,4093.379883,4111.080078,4114240000,0.0,0.0


In [72]:
nasdaq = yf.Ticker('^IXIC')
nasdaq_history = sp500.history(start=start_day)

nasdaq_history.reset_index(inplace=True)
nasdaq_history['Date'] = nasdaq_history['Date'].dt.strftime('%Y-%m-%d')
nasdaq_history.set_index(nasdaq_history['Date'], inplace=True)
nasdaq_history.drop(['Date'], axis=1, inplace=True)

nasdaq_history.rename({'Close':'Nasdaq'}, axis=1, inplace=True)

In [73]:
data = pd.concat([IBM_history, sp500_history['SP500'].round(2), nasdaq_history['Nasdaq'].round(2)], axis=1)

In [74]:
data

,Open,High,Low,Close,Volume,Rate,MA_20,Upper Band,Lower Band,SP500,Nasdaq
Date,,,,,,,,,,,
1999-01-04,50.837228,51.249422,49.875442,50.287636,8524482,-1.08,50,50.55,49.45,1228.10,1228.10
1999-01-05,50.287637,52.176859,50.236115,52.108158,10363350,3.62,51,53.57,48.43,1244.78,1244.78
1999-01-06,52.297049,52.966864,51.798983,51.867680,9978422,-0.46,51,52.98,49.02,1272.34,1272.34
1999-01-07,51.644467,52.863875,51.386846,52.262756,8688913,0.76,51,52.82,49.18,1269.73,1269.73
1999-01-08,52.486001,52.760797,51.008972,51.541389,9598933,-1.38,51,52.58,49.42,1275.09,1275.09
...,...,...,...,...,...,...,...,...,...,...,...
2023-02-02,135.960007,136.720001,134.850006,136.389999,6107800,0.96,140,148.22,131.78,4179.76,4179.76
2023-02-03,136.350006,136.949997,135.529999,136.940002,3754500,0.40,140,148.38,131.62,4136.48,4136.48
2023-02-06,135.830002,136.320007,134.949997,136.179993,4841300,-0.55,140,148.43,131.57,4111.08,4111.08


In [ ]:
from keras